https://link.springer.com/content/pdf/10.1007/s11075-012-9599-1.pdf

In [239]:
%load_ext line_profiler

In [2]:
import torch
import numpy as np
from torch_sparse import SparseTensor

In [ ]:
from data.dataset import LPDataset
import torch
import numpy as np
from tqdm import tqdm
from torch_sparse import SparseTensor

ds = torch.load('datasets/large_mis/processed/batch0.pt')

In [ ]:
g = ds[i]
A = SparseTensor(row=g['cons', 'to', 'vals'].edge_index[0],
                 col=g['cons', 'to', 'vals'].edge_index[1],
                 value=g['cons', 'to', 'vals'].edge_attr.squeeze(),
                 is_sorted=True, trust_data=True).to_dense().numpy()

In [ ]:
def density(x):
    return np.nonzero(x)[0].shape[0] / np.prod(x.shape)

In [672]:
from scipy.sparse import csr_matrix, coo_matrix


def sparse_null_space(A: np.ndarray):
    nnz_rows = np.where(A)[0]
    row_idx, counts = np.unique(nnz_rows, return_counts=True)
    sort_idx = np.argsort(counts)
    A_sorted = A[sort_idx, :]

    m, n = A.shape
    H = np.eye(n)
    
    for i in range(m):
        s_i = H.dot(A_sorted[i])
        if np.all(s_i == 0.):
            continue
        ji = np.argmax(np.abs(s_i)[:n - i])
        si_prime = - s_i / s_i[ji]
        si_prime = np.hstack([si_prime[:ji], si_prime[ji+1:]])
        H_new = np.zeros((si_prime.shape[0], n), dtype=np.float32)
        nnz = np.where(si_prime)[0]
        H_new[nnz, :] = si_prime[nnz, None] * H[ji : ji+1, :]
        H_new[:ji] += H[:ji]
        H_new[ji:] += H[ji+1:]
        H = H_new
    return H.T

In [673]:
# from scipy.sparse import csr_matrix, coo_matrix, eye as sp_eye, vstack as sp_vstack


# def sparse_null_space(A: np.ndarray):
#     nnz_rows = np.where(A)[0]
#     row_idx, counts = np.unique(nnz_rows, return_counts=True)
#     sort_idx = np.argsort(counts)
#     A_sorted = csr_matrix(A[sort_idx, :])

#     m, n = A.shape
#     H = sp_eye(n, format='csr')
    
#     for i in range(m):
#         s_i = H @ A_sorted[i].T
#         if np.all(s_i.data == 0.) or len(s_i.data) == 0.:
#             continue
#         ji = np.argmax(np.abs(s_i)[:n - i])
#         si_prime = - s_i / s_i[ji].data.item()
#         si_prime = sp_vstack([si_prime[:ji], si_prime[ji+1:]])
#         H_new = si_prime * H[ji, :]
#         H_new[:ji] += H[:ji]
#         H_new[ji:] += H[ji+1:]
#         H = H_new
#     return H.T

In [674]:
H = sparse_null_space(A)

In [675]:
np.allclose(A @ H, 0.)

True

In [676]:
%lprun -f sparse_null_space sparse_null_space(A)

Timer unit: 1e-09 s

Total time: 0.143467 s
File: /var/folders/6v/m172f7bs02l0tkwy4_9yxttm0000gn/T/ipykernel_86621/3000469663.py
Function: sparse_null_space at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def sparse_null_space(A: np.ndarray):
     5         1    6957000.0    7e+06      4.8      nnz_rows = np.where(A)[0]
     6         1     315000.0 315000.0      0.2      row_idx, counts = np.unique(nnz_rows, return_counts=True)
     7         1      35000.0  35000.0      0.0      sort_idx = np.argsort(counts)
     8         1     719000.0 719000.0      0.5      A_sorted = A[sort_idx, :]
     9                                           
    10         1       2000.0   2000.0      0.0      m, n = A.shape
    11         1     709000.0 709000.0      0.5      H = np.eye(n)
    12                                               
    13       647      92000.0    142.2      0.1      for i in range(m):
    14       646   

In [677]:
q, r = np.linalg.qr(H)

In [678]:
np.abs(A @ q).max()

2.9802322e-08

In [680]:
density(H)

0.006807684232626496

In [681]:
density(q)

0.6590909090909091

In [682]:
pj = H @ H.T

In [683]:
density(pj)

0.013558128743552954

In [684]:
x = np.random.rand(H.shape[0])

In [685]:
x1 = pj @ x

In [686]:
x.dot(x1) / np.linalg.norm(x, ord=2.) / np.linalg.norm(x1, ord=2.)

0.5197014786043137

In [687]:
pj2 = q @ q.T

In [688]:
density(pj2)

1.0

In [689]:
x2 = pj2 @ x

In [690]:
x.dot(x2) / np.linalg.norm(x, ord=2.) / np.linalg.norm(x2, ord=2.)

0.5954144780351396

In [691]:
proj = H @ (H / (H * H).sum(0)).T

In [692]:
density(proj)

0.013558128743552954

In [693]:
x3 = proj @ x

In [694]:
x.dot(x3) / np.linalg.norm(x, ord=2.) / np.linalg.norm(x3, ord=2.)

0.5832242564961544

In [695]:
from torch_geometric.data import Batch

In [697]:
for pkg in range(10):
    new_g = []
    ds = torch.load(f'datasets/large_mis/processed/batch{pkg}.pt')
    for i in tqdm(range(len(ds))):
        g = ds[i]
        A = SparseTensor(row=g['cons', 'to', 'vals'].edge_index[0],
                     col=g['cons', 'to', 'vals'].edge_index[1],
                     value=g['cons', 'to', 'vals'].edge_attr.squeeze(),
                     is_sorted=True, trust_data=True).to_dense().numpy()

        H = sparse_null_space(A)
        proj = H @ (H / (H * H).sum(0)).T
    
        wh = np.vstack(np.where(proj))
        proj_edge_index = torch.from_numpy(wh).long()
        proj_val = torch.from_numpy(proj[wh[0], wh[1]]).float()
    
        data = HeteroData(
                    cons={
                        'num_nodes': g.b.shape[0],
                         },
                    vals={
                        'num_nodes': g.c.shape[0],
                    },
                
                    cons__to__vals={'edge_index': g['cons', 'to', 'vals'].edge_index,
                                    'edge_attr': g['cons', 'to', 'vals'].edge_attr},
    
                    vals__to__vals={'edge_index': proj_edge_index,
                                    'edge_attr': proj_val},
    
                    x_solution=g.x_solution,
                    x_feasible=g.x_feasible,
                    obj_solution=g.obj_solution,
                    c=g.c,
                    b=g.b,
                )
    
        new_g.append(data)
    torch.save(Batch.from_data_list(new_g), f'datasets/large_mis_sp/processed/batch{pkg}.pt')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:36<00:00,  6.38it/s]
